<a href="https://colab.research.google.com/github/tomonari-masada/course2024-intro2ml/blob/main/12_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 多層パーセプトロン(MLP; multi-layer perceptron)
* シンプルなニューラルネットワークである。
 * CNNやRNNのようなアーキテクチャに比べてシンプル、ということ。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'

## toy dataでMLPを試す

In [ ]:
from sklearn.neural_network import MLPClassifier

X_xor = [[0., 0.], [1., 1.], [0., 1.], [1., 0.]]
y_xor = [0, 0, 1, 1]
clf = MLPClassifier(random_state=1)
clf.fit(X_xor, y_xor)

* どういうデータを分類していたかを確認
 * 線形なモデルは学習できない問題。

In [ ]:
X_xor = np.array([[0., 0.], [1., 1.], [0., 1.], [1., 0.]])
plt.scatter(X_xor[:,0], X_xor[:,1], c=y_xor)
plt.xlim(-.5,1.5)
plt.ylim(-.5,1.5)
plt.show()

### 分類境界を可視化する
* 直線ではなく、曲線で分割できていることに注意。

In [ ]:
x1 = np.linspace(-0.5, 1.5, 101)
x2 = np.linspace(-0.5, 1.5, 101)
z = clf.predict(np.array([[i, j] for j in x2 for i in x1])).reshape(101,101)
plt.figure(figsize=(9,9))
plt.contourf(x1, x2, z, 100, alpha=0.1)
plt.scatter(X_xor[:,0], X_xor[:,1], c=y_xor)
plt.show()

## MNISTデータを使う

* scikit-learnに用意されている仕組みを利用してMNISTデータをダウンロードする。

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
X, y = mnist.data, mnist.target
print(X.shape)
print(y.shape)

* 通常、MNISTデータは最後の10000件をテストデータとして使う。

In [ ]:
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

* テストデータ以外の60000件を、訓練データ50000件と検証データ10000件へ分割しておく。

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train,
                                                      test_size=10000,
                                                      random_state=42)

* 訓練データの最初の100個を可視化してみる。

In [ ]:
fig, axes = plt.subplots(10, 10)
for x, ax in zip(X_train.values, axes.ravel()):
  ax.axis('off') # 軸を消す
  ax.matshow(x.reshape(28, 28), cmap=plt.cm.gray)
plt.show()

* 今回は、「０」から「９」の10種類へと分類する10値分類の問題を解く。
 * 前回までのように2値分類に変換したりはしない。

* 実は、今までのロジスティック回帰やSVMも、2値分類に変換しなければ、10値分類として実行できていた！
 * 各自確認してみてください。

## 前処理：標準化
* 各ピクセルでの平均が0、標準偏差が1になるように、データを標準化しておく。
* こうすると分類精度が良くなったりする。

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train) # 訓練データで平均と標準偏差を計算
X_train = scaler.transform(X_train) # 訓練データで求めた平均と標準偏差を使って標準化する
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)


## MLPによる分類の実行


* MLPを使う準備
 * いろいろな設定項目を指定する。

In [ ]:
clf = MLPClassifier(solver='adam', alpha=1e-5,
                    hidden_layer_sizes=100, random_state=1,
                    verbose=True, # 損失関数の値を表示
                    warm_start=True) # 学習を再開できるようにする

* 待てないときは、強制的に止めることもできる。（学習結果は残っている。）

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
print(f'Accuracy: {clf.score(X_valid, y_valid):.4f}')

* どの数字をどのクラスに何個間違ったかを表す混同行列を可視化する。

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_valid, clf.predict(X_valid))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
cm = confusion_matrix(y_valid, clf.predict(X_valid))
cm_display = ConfusionMatrixDisplay(cm).plot()

## 見つけ出した最善の設定を使ってテストデータ上で評価


In [ ]:
clf = MLPClassifier(solver='adam', alpha=1e-5,
                    hidden_layer_sizes=100, random_state=1,
                    verbose=True,
                    warm_start=True)

In [ ]:
X_train_valid = np.concatenate([X_train, X_valid])

In [ ]:
y_train_valid = np.concatenate([y_train, y_valid])

In [ ]:
clf.fit(X_train_valid, y_train_valid)

In [ ]:
print(f'Test Accuracy: {clf.score(X_test, y_test):.4f}')